In [17]:
import random
import requests
import csv
import traceback
import time
import random
from concurrent.futures import ThreadPoolExecutor
from itertools import islice
from urllib.error import URLError, HTTPError
from urllib.request import urlopen
import os
import shutil

class ProductIdsCrawling:
    def __init__(self):
        self.product_ids_file_path = './data/product_ids_v4.csv'
        self.categories_file_path = './data/category_ids.csv'
        self.user_agent_file_path = './data/user-agents.txt'
        self.api_errors = './data/api_error.csv'
        self.base_link = 'https://tiki.vn'
        self.api_menu = 'https://api.tiki.vn/raiden/v2/menu-config?'
        self.api_sub_menu = 'https://tiki.vn/api/personalish/v1/blocks/listings?'
        self.user_agents = []
        self.proxies = []
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
            'Accept': 'application/json, text/plain, */*',
            'Accept-Language': 'en-US,en;q=0.9,vi-VN;q=0.8,vi;q=0.7',
            'x-guest-token': 'noUm6Bg7hPXcdYtiHREzDfxIOClk1Nup',
            'Connection': 'keep-alive',
            'TE': 'Trailers',
        }
        self.params_api = {
            'limit': '100',
            'include': 'advertisement',
            'aggregations': '2',
            'trackity_id': '5a387aad-bb84-2dca-2df9-946cc84ccf0f',
            'page': '1'
        }

    def read_user_agents(self):
        with open(self.user_agent_file_path, 'r') as file:
            for line in file:
                self.user_agents.append(line.strip())
            
    def store_api_errors(self, params):
         with open(self.api_errors, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:
                writer.writerow(['limit', 'inclue', 'aggregations', 'trackity_id', 'page', 'url', 'category'])
            writer.writerow([params['limit'],params['include'],params['aggregations'],params['trackity_id'],params['page'],params['url'],params['category']])

    def read_categories(self):
        with open(self.categories_file_path, 'r') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            rows_to_skip = 1
            rows = islice(spamreader, rows_to_skip, None)
            for row in rows:
                link = row[0].split(',')
                url = link[0]
                category_id = link[1]
                self.product_id_crawling(url, category_id, 1)
            csvfile.close()
            
    def store_products_ids(self, product_id, filepath):
        try:
            with open(filepath, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                if csvfile.tell() == 0:
                    writer.writerow(['id'])
                writer.writerow([product_id])
        except (URLError, HTTPError, Exception) as e:
            print(f"Connect to api error{self.params_api} with {e}")
            self.store_api_errors(self.params_api)

    def save_ids_to_txt(ids, file_path):
        with open(file_path, 'w') as file:
            for id in ids:
                file.write(str(id) + '\n')

    def product_id_crawling(self, url, category_id, page):
        self.params_api['category'] = category_id
        self.params_api['url'] = url
        self.params_api['page'] = page
        self.headers['User-Agent'] = random.choice(self.user_agents)
        try:
            response = requests.get(
                self.api_sub_menu,
                headers=self.headers,
                params=self.params_api
            )
            if response.status_code == 200:
                data = response.json().get("data", "")
                if data:
                    for item in data:
                        product_id = item.get("id", "")
                        self.store_products_ids(product_id, self.product_ids_file_path)
                    last_page = response.json().get("paging", {}).get("last_page")
                    current_page = response.json().get("paging", {}).get("current_page")
                    if int(current_page) % 5 == 0:
                        time.sleep(random.randrange(5, 10))
                    if last_page > current_page:
                        self.product_id_crawling(url, category_id, current_page + 1)
            else:
                print(f"{response.status_code} Connect to api error{self.params_api}")
                self.store_api_errors(self.params_api)
        except (URLError, HTTPError, Exception) as e:
            print(f"Connect to api error{self.params_api} with {e}")
            self.store_api_errors(self.params_api)
            

    def main(self):
        print("Started")
        self.read_user_agents()
        self.read_categories()
        print("Succeeded")

if __name__ == "__main__":
    crawler = ProductIdsCrawling()
    crawler.main()


Started
500 Connect to api error{'limit': '100', 'include': 'advertisement', 'aggregations': '2', 'trackity_id': '5a387aad-bb84-2dca-2df9-946cc84ccf0f', 'page': 14, 'category': '2250', 'url': 'sach-tham-khao-cap-iii'}
500 Connect to api error{'limit': '100', 'include': 'advertisement', 'aggregations': '2', 'trackity_id': '5a387aad-bb84-2dca-2df9-946cc84ccf0f', 'page': 1, 'category': '7971', 'url': 'sach-luyen-thi-dai-hoc-cao-dang'}
500 Connect to api error{'limit': '100', 'include': 'advertisement', 'aggregations': '2', 'trackity_id': '5a387aad-bb84-2dca-2df9-946cc84ccf0f', 'page': 3, 'category': '9997', 'url': 'the-luyen-thi'}
Connect to api error{'limit': '100', 'include': 'advertisement', 'aggregations': '2', 'trackity_id': '5a387aad-bb84-2dca-2df9-946cc84ccf0f', 'page': 7, 'category': '26568', 'url': 'am-thanh-phu-kien-tivi'} with ('Connection broken: IncompleteRead(2774 bytes read, 1322 more expected)', IncompleteRead(2774 bytes read, 1322 more expected))
Connect to api error{'lim